### Step 1: Reading and Understanding the Data

In [4]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Importing all required packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from sklearn.impute import KNNImputer



In [6]:
# Importing dataset

lead = pd.read_csv('research_joint_data.csv', encoding = 'latin')
lead.head()

,Id,StageName,Status_Reason__c,RecordType.Name,RICE_Supported__c,CreatedDate,CloseDate,Actual_Close_Date__c,AccountId,Lead_Faculty__c,Lead_School__c,Parent_Opportunity__c,RecordType.Name.1,Industry,Industry_Sub_Type__c,Business_Type__c,Is_External__c,ParentId
0,0062e000002Hc2PAAS,Closed Lost,Customer No Longer Interested,Custom Education (MSPACE Included),RIC-BD&I,2019-11-07T00:59:46.000Z,31/8/20,30/4/20,0012e000003AqeVAAS,0012e000002ZGfbAAG,0012e000002Zt0mAAC,NaN,Business Organization,Health,Health Care & Healthy Aging,Multinational / Other Large Corporate,External,NaN
1,0062e000002HFaaAAG,Post Award,NaN,Parent Grant,RIC-RE&D,2018-09-19T04:32:55.000Z,31/12/49,NaN,0012e000003A6ElAAK,NaN,NaN,NaN,Business Organization,Public Administration,NaN,Government: Australia: Federal,External,0012e000003A6osAAC
2,0062e000002HFabAAG,Post Award,NaN,Parent Grant,RIC-RE&D,2018-09-19T04:32:19.000Z,22/6/20,22/6/20,0012e000003A6ElAAK,NaN,NaN,NaN,Business Organization,Public Administration,NaN,Government: Australia: Federal,External,0012e000003A6osAAC
3,0062e000002HFacAAG,Closed,NaN,Parent Grant,NaN,2016-07-06T05:03:00.000Z,31/12/49,NaN,0012e000003A6osAAC,NaN,NaN,NaN,Business Organization,Public Administration,NaN,Government: Australia: Federal,External,0012e000003A6FfAAK
4,0062e000002HFadAAG,Closed,NaN,Parent Grant,NaN,2016-07-21T23:56:54.000Z,31/12/49,NaN,0012e000003A79XAAS,NaN,NaN,NaN,Business Organization,Public Administration,NaN,Government: Australia: Federal,External,NaN


In [7]:
# inspect lead dataframe

print("*********************************  Info *******************************************") 
print(lead.info())
print("*********************************  Shape ******************************************") 
print(lead.shape)
print("**************************** Columns having null values *****************************")
print(lead.isnull().any())
print("**************************** Describe *****************************")
lead.describe()

*********************************  Info *******************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7597 entries, 0 to 7596
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Id                     7597 non-null   object
 1   StageName              7597 non-null   object
 2   Status_Reason__c       7288 non-null   object
 3   RecordType.Name        7597 non-null   object
 4   RICE_Supported__c      7569 non-null   object
 5   CreatedDate            7597 non-null   object
 6   CloseDate              7597 non-null   object
 7   Actual_Close_Date__c   6293 non-null   object
 8   AccountId              6840 non-null   object
 9   Lead_Faculty__c        6325 non-null   object
 10  Lead_School__c         4272 non-null   object
 11  Parent_Opportunity__c  982 non-null    object
 12  RecordType.Name.1      6840 non-null   object
 13  Industry               6838 non-null   

,Id,StageName,Status_Reason__c,RecordType.Name,RICE_Supported__c,CreatedDate,CloseDate,Actual_Close_Date__c,AccountId,Lead_Faculty__c,Lead_School__c,Parent_Opportunity__c,RecordType.Name.1,Industry,Industry_Sub_Type__c,Business_Type__c,Is_External__c,ParentId
count,7597,7597,7288,7597,7569,7597,7597,6293,6840,6325,4272,982,6840,6838,4199,6795,6840,1140
unique,7597,18,25,12,4,7498,885,1072,2408,16,46,315,3,19,19,11,2,105
top,0062e000002Y1QJAA0,Closed Lost,Other (Lost),Research Contract,RIC-BD&I,2016-08-08T06:48:56.000Z,1/11/2019,11/5/2017,0012e000003A6qHAAS,0012e000002ZGfbAAG,0012e000002Zt0cAAC,0062e000002HFbAAAW,Business Organization,Services,Education,Government: Australia: Federal,External,0012e000003A78yAAC
freq,1,4003,2398,3453,3289,14,2726,103,324,1514,526,50,6796,2457,1366,1419,6796,319


# Step 2: Data Cleaning

In [8]:
#percentage of null values for each column

percent_missing = round(100*(lead.isnull().sum()/len(lead.index)), 2)
print(percent_missing)

Id                        0.00
StageName                 0.00
Status_Reason__c          4.07
RecordType.Name           0.00
RICE_Supported__c         0.37
CreatedDate               0.00
CloseDate                 0.00
Actual_Close_Date__c     17.16
AccountId                 9.96
Lead_Faculty__c          16.74
Lead_School__c           43.77
Parent_Opportunity__c    87.07
RecordType.Name.1         9.96
Industry                  9.99
Industry_Sub_Type__c     44.73
Business_Type__c         10.56
Is_External__c            9.96
ParentId                 84.99
dtype: float64


There are some columns that have more than 40% null values.

remove: (Lead_School__c  43.77, Industry_Sub_Type__c     44.73)

ParentId: 1yes/0no

Parent_Opportunity__c: 1yes/0no

###### Check if there are any duplicate values in the dataset

In [9]:
lead[lead.duplicated(keep=False)]

,Id,StageName,Status_Reason__c,RecordType.Name,RICE_Supported__c,CreatedDate,CloseDate,Actual_Close_Date__c,AccountId,Lead_Faculty__c,Lead_School__c,Parent_Opportunity__c,RecordType.Name.1,Industry,Industry_Sub_Type__c,Business_Type__c,Is_External__c,ParentId


There are no duplicate values in the data.

###### Transform to 1yes/0no

In [10]:
lead[["ParentId","Parent_Opportunity__c"]]=lead[["ParentId","Parent_Opportunity__c"]].notnull().astype(int)

###### Drop 3 columns

In [11]:
lead = lead.drop(["Lead_School__c","Industry_Sub_Type__c"],axis=1)

In [12]:
 lead = lead.drop(["Actual_Close_Date__c"],axis=1)

In [13]:
#check null values
percent_missing = round(100*(lead.isnull().sum()/len(lead.index)), 2)
print(percent_missing)

Id                        0.00
StageName                 0.00
Status_Reason__c          4.07
RecordType.Name           0.00
RICE_Supported__c         0.37
CreatedDate               0.00
CloseDate                 0.00
AccountId                 9.96
Lead_Faculty__c          16.74
Parent_Opportunity__c     0.00
RecordType.Name.1         9.96
Industry                  9.99
Business_Type__c         10.56
Is_External__c            9.96
ParentId                  0.00
dtype: float64


###### Impute the null values by the median value:

Industry

Status_Reason__c

RICE_Supported__c 

RecordType.Name.1 

Business_Type__c

##### Impute the null values by K-means:
AccountId

Lead_Faculty__c




In [14]:
lead['Industry'].value_counts()


Services                                    2457
Health                                      1530
Public Administration                        963
Defence                                      477
Technology: IT & Telephone Services          263
Agriculture & Horticulture                   259
Manufacturing & R&D                          212
Energy & Utilities                           191
Construction & Construction Materials        137
Transportation & Logistics                   108
Mining & Refining                             73
Materials: Chemicals & Industrial Metals      50
Education                                     43
Veterinary                                    35
Mining Equipment and Technology Services      17
Transportation                                 8
Forestry & Paper                               6
Aquaculture & Fisheries                        6
Advertising, Marketing and PR                  3
Name: Industry, dtype: int64

In [15]:
# since "Services" has the major chunk of data, 
# we can impute the null values with Services
lead['Industry'] = lead['Industry'].replace(np.nan, "Services")

In [16]:
lead['Status_Reason__c'].value_counts()

Other (Lost)                         2398
Won                                  2014
On Track                              907
Customer No Longer Interested         720
Proposal/Application Unsuccessful     442
Academic Capacity (Lost)              215
Out-Sold                              139
Customer Not Responding               103
Awaiting Customer Action               95
Academic No Longer Interested          79
Not Ready for Market                   48
BD Capacity (Lost)                     29
Failure to Agree - Price               14
Failure to Agree - Scope of Work       14
Academic Capacity (Open)               13
Unacceptable Risk                      11
Risk                                    8
Other (Open)                            8
Price                                   8
Other IP Issue                          7
Scope of Proposal                       6
Failure to Agree - IP terms             5
IP Issue                                3
BD Capacity (Open)                

In [17]:
lead['Status_Reason__c'] = lead['Status_Reason__c'].replace(np.nan, "Other (Lost)")

In [18]:
lead['RICE_Supported__c'].value_counts()

RIC-BD&I             3289
RIC RE&D and BD&I    3084
Not supported         637
RIC-RE&D              559
Name: RICE_Supported__c, dtype: int64

In [19]:
lead['RICE_Supported__c'] = lead['RICE_Supported__c'].replace(np.nan, "RIC-BD&I")

In [20]:
lead['RecordType.Name.1'].value_counts()

Business Organization    6796
University Department      43
Administrative              1
Name: RecordType.Name.1, dtype: int64

In [21]:
lead['RecordType.Name.1'] = lead['RecordType.Name.1'].replace(np.nan, "Business Organization")

In [22]:
lead['Business_Type__c'].value_counts()

Government: Australia: Federal                       1419
Government: Australia: Local & State                 1288
SME (small to medium enterprise)                      914
Multinational / Other Large Corporate                 853
Not for profit                                        823
University                                            535
Large Australian Corporate                            440
Government: International                             321
PFRO (Publicly-Funded Research Organisation)          101
RDC (Rural Research and Development Corporations)      53
CRC (Co-operative Research Centre)                     48
Name: Business_Type__c, dtype: int64

In [23]:
lead['Business_Type__c'] = lead['Business_Type__c'].replace(np.nan, "Not for profit")

 ##### k means clustering doesn't work
 could not convert string to float

 AccountId

Lead_Faculty__c

imputer = KNNImputer(n_neighbors=2)
lead_filled = imputer.fit_transform(lead)

In [24]:
lead['Lead_Faculty__c'].value_counts()

0012e000002ZGfbAAG    1514
0012e000002ZLg7AAG    1204
0012e000002Zmo0AAC     875
0012e000002ZmnxAAC     845
0012e000002ZLgJAAW     359
0012e000002ZmnzAAC     357
0012e000002ZNYOAA4     355
0012e000002ZmOZAA0     321
0012e000002ZLgIAAW     294
0012e000002Zt1RAAS     172
0012e000002ZFZYAA4       9
0012e000002ZmOUAA0       7
0012e000003hllpAAA       5
0012e000002Zt0MAAS       5
0012e000002Zt1zAAC       2
0012e000002Zt1iAAC       1
Name: Lead_Faculty__c, dtype: int64

In [25]:
#only closed won closed loss is remained

lead[lead.StageName != 'Closed Lost' or lead.StageName != 'Closed Won' ]


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [73]:
lead.to_csv("cleaned.csv",index=False)